In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 113 rows, 113 columns and 113 nonzeros
Model has 6441 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-02, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 113 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 113 columns, 0 nonzeros
Presolved model has 6441 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 23
 AA' NZ     : 2.530e+02
 Factor NZ  : 2.760e+02
 Factor Ops : 4.324e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.71803846e+06 -2.03792550e+05  2.08e+04 1.20e+02  9.68e+05     0s
   1   4.37286791e+06 -5.14826560e+06  1.91e+03 2.27e-13  1.71e+05     0s
   2   1.21811766e+06 -1.53553095e+06  1.91e-03 1.14e-13  2.44e+04     0s
   3   1.1607

In [2]:
xi_list = GLS_Apr_weekday_PM.xi_list
P = GLS_Apr_weekday_PM.P
L = GLS_Apr_weekday_PM.L  # dimension of xi

113

In [3]:
size(P, 1), size(P, 2)

(56,113)

In [4]:
P

56x113 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [5]:
xi_list

113-element Array{Any,1}:
 395.97       
 110.164      
   2.1431e-11 
 449.967      
 128.839      
   3.89738e-11
   2.36189e-11
   5.36267e-11
   2.43875e-11
   4.76849e-11
  65.6625     
   2.91491e-11
  78.8928     
   ⋮          
   1.34539e-11
   5.20024e-11
   4.50559e-11
 639.376      
   5.82389e-11
   1.26517e-10
   3.04204e-10
   4.25772e-11
   4.10293e-11
   1.70278e-10
 253.183      
   1.16409e-10

In [6]:
using JuMP

mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    25199
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    18984

Total number of variables............................:     6384
                     variables with only lower bounds:     6384
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [7]:
getvalue(lam)

56-element Array{Float64,1}:
  395.97      
  110.164     
  578.806     
    6.25557e-9
   65.6625    
   78.8928    
  146.84      
    2.50188e-9
    2.47927e-9
    6.21923e-9
  224.375     
    2.58196e-9
    6.26941e-9
    ⋮         
 1174.34      
    6.19556e-9
  209.637     
   79.8991    
    2.47478e-9
 1309.79      
  639.376     
    7.22923e-9
    2.50406e-9
    2.50271e-9
  253.183     
    2.56858e-9

In [8]:
getobjectivevalue(mGLSJulia)

1.0000000000000089

In [9]:
GLS_Apr_weekday_PM.saveDemandVec(getvalue(lam))

In [10]:
getvalue(p)

56x113 Array{Float64,2}:
 1.0          0.0          0.0          …  0.0          0.0        
 2.27951e-15  1.0          6.17141e-12     1.24586e-15  0.0        
 0.0          0.0          0.0             0.0          0.0        
 2.10489e-16  2.10489e-16  2.10489e-16     2.10489e-16  2.10489e-16
 1.63575e-15  4.02546e-16  2.89389e-16     1.01965e-15  0.0        
 1.85433e-15  3.72646e-16  2.36514e-16  …  1.11409e-15  0.0        
 2.6144e-15   0.0          0.0             1.23663e-15  0.0        
 2.10532e-16  2.10532e-16  2.10532e-16     2.10532e-16  2.10532e-16
 2.10532e-16  2.10532e-16  2.10532e-16     2.10532e-16  2.10532e-16
 2.10489e-16  2.10489e-16  2.10489e-16     2.10489e-16  2.10489e-16
 2.74056e-15  0.0          0.0          …  6.35294e-16  0.0        
 2.10532e-16  2.10532e-16  2.10532e-16     2.10532e-16  2.10532e-16
 2.10489e-16  2.10489e-16  2.10489e-16     2.10489e-16  2.10489e-16
 ⋮                                      ⋱                          
 0.0          0.0      

In [11]:
route_choice_matrix_Apr_PM = getvalue(p)

56x113 Array{Float64,2}:
 1.0          0.0          0.0          …  0.0          0.0        
 2.27951e-15  1.0          6.17141e-12     1.24586e-15  0.0        
 0.0          0.0          0.0             0.0          0.0        
 2.10489e-16  2.10489e-16  2.10489e-16     2.10489e-16  2.10489e-16
 1.63575e-15  4.02546e-16  2.89389e-16     1.01965e-15  0.0        
 1.85433e-15  3.72646e-16  2.36514e-16  …  1.11409e-15  0.0        
 2.6144e-15   0.0          0.0             1.23663e-15  0.0        
 2.10532e-16  2.10532e-16  2.10532e-16     2.10532e-16  2.10532e-16
 2.10532e-16  2.10532e-16  2.10532e-16     2.10532e-16  2.10532e-16
 2.10489e-16  2.10489e-16  2.10489e-16     2.10489e-16  2.10489e-16
 2.74056e-15  0.0          0.0          …  6.35294e-16  0.0        
 2.10532e-16  2.10532e-16  2.10532e-16     2.10532e-16  2.10532e-16
 2.10489e-16  2.10489e-16  2.10489e-16     2.10489e-16  2.10489e-16
 ⋮                                      ⋱                          
 0.0          0.0      

In [12]:
sum(route_choice_matrix_Apr_PM[1, :])

1.0000000000039524

In [13]:
route_choice_matrix_dict_Apr_PM = Dict()

for row_idx = 1:size(route_choice_matrix_Apr_PM)[1]
    for col_idx = 1:size(route_choice_matrix_Apr_PM)[2]
        route_choice_matrix_dict_Apr_PM[(row_idx, col_idx)] = route_choice_matrix_Apr_PM[row_idx, col_idx]
    end
end

In [20]:
route_choice_matrix_dict_Apr_PM

Dict{Any,Any} with 6328 entries:
  (21,13)  => 2.104885204232967e-16
  (53,24)  => 2.105323322651197e-16
  (23,106) => 0.0
  (18,103) => 5.880610928199881e-15
  (33,111) => 2.1053233230360348e-16
  (41,34)  => 0.0
  (32,65)  => 2.1053233185494988e-16
  (56,55)  => 2.1053233176496925e-16
  (54,81)  => 2.1053233223882853e-16
  (24,108) => 2.1048851986501038e-16
  (16,73)  => 2.1053233203802872e-16
  (1,89)   => 0.0
  (16,67)  => 2.1053233185784027e-16
  (10,85)  => 2.1048852115958793e-16
  (40,97)  => 0.0
  (1,22)   => 0.0
  (24,31)  => 2.1048851984649424e-16
  (6,41)   => 9.431632640223406e-16
  (37,55)  => 2.105323317920682e-16
  (34,113) => 2.1053233182600815e-16
  (19,26)  => 0.0
  (50,27)  => 0.0
  (36,25)  => 2.1048852051039958e-16
  (9,108)  => 2.105323318804711e-16
  (33,33)  => 2.105323318601753e-16
  ⋮        => ⋮

In [ ]:
outfile = open("../temp_files/route_choice_CDC_Apr_PM.json", "w")

JSON.print(outfile, route_choice_matrix_dict_Apr_PM)

close(outfile)